# Predictive Modelling using Machine Learning Algorithms

In [1]:
import pandas as pd
import numpy as np
import duckdb as db
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import optuna
import warnings

warnings.filterwarnings('ignore')

from IPython.display import display, Markdown
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# pd.set_option('plotting.backend', 'plotly')
# pio.renderers.default = "notebook_connected"
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [14]:
orig = pd.read_parquet("../../data/processed/train_enhanced.parquet")

In [21]:
orig

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Orders,Sales,...,Day,Day_of_Week,Month_Name,Month,Year,Quarter,Week,Week_of_Month,Is_Weekend,cluster
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,...,1,0,January,1,2018,1,1,1,0,3
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,...,1,0,January,1,2018,1,1,1,0,2
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,...,1,0,January,1,2018,1,1,1,0,1
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,...,1,0,January,1,2018,1,1,1,0,4
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,...,1,0,January,1,2018,1,1,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,...,31,4,May,5,2019,2,22,5,0,4
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,...,31,4,May,5,2019,2,22,5,0,2
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,...,31,4,May,5,2019,2,22,5,0,3
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41,...,31,4,May,5,2019,2,22,5,0,1


In [39]:
print("Min date:", orig["Date"].min(), ", Max date:", orig["Date"].max())

total_days = (orig["Date"].max() - orig["Date"].min()).days
train_max = orig["Date"].min() + pd.DateOffset(days=total_days * 0.8)
print("Total days:", total_days, ", Training days: ",total_days * 0.8 )
print("Train max date:", train_max)


Min date: 2018-01-01 00:00:00 , Max date: 2019-05-31 00:00:00
Total days: 515 , Training days:  412.0
Train max date: 2019-02-17 00:00:00


In [16]:
orig.columns

Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Date',
       'Holiday', 'Discount', 'Orders', 'Sales', 'Day_of_Week_Name', 'Day',
       'Day_of_Week', 'Month_Name', 'Month', 'Year', 'Quarter', 'Week',
       'Week_of_Month', 'Is_Weekend', 'cluster'],
      dtype='object')

In [40]:
columns = ['Store_id', 'Store_Type', 'Location_Type', 'Region_Code',
       'Holiday', 'Discount',  'Sales', 'Day', 'Date',
       'Day_of_Week', 'Month', 'Quarter', 'Week',
       'Week_of_Month', 'Is_Weekend', "cluster"]

In [41]:
df=orig[columns]

In [61]:
train = df[df["Date"] <= train_max]
test = df[df["Date"] > train_max]

In [62]:
train

,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount,Sales,Day,Date,Day_of_Week,Month,Quarter,Week,Week_of_Month,Is_Weekend,cluster
0,1,S1,L3,R1,1,1,7011.84,1,2018-01-01,0,1,1,1,1,0,3
1,253,S4,L2,R1,1,1,51789.12,1,2018-01-01,0,1,1,1,1,0,2
2,252,S3,L2,R1,1,1,36868.20,1,2018-01-01,0,1,1,1,1,0,1
3,251,S2,L3,R1,1,1,19715.16,1,2018-01-01,0,1,1,1,1,0,4
4,250,S2,L3,R4,1,1,45614.52,1,2018-01-01,0,1,1,1,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150740,33,S4,L2,R4,0,1,73200.00,17,2019-02-17,6,2,1,7,3,1,2
150741,254,S4,L1,R1,0,1,55842.00,17,2019-02-17,6,2,1,7,3,1,2
150742,109,S1,L1,R1,0,1,52578.00,17,2019-02-17,6,2,1,7,3,1,3
150743,303,S4,L2,R1,0,1,69456.00,17,2019-02-17,6,2,1,7,3,1,2


In [63]:
train=db.sql("""
    select *,
    lag(Sales, 1) over (partition by Store_id order by Date) as Sales_Lag_1,
    lag(Sales, 7) over (partition by Store_id order by Date) as Sales_Lag_7,
    lag(Sales, 12) over (partition by Store_id order by Date) as Sales_Lag_12,
    lag(Sales, 30) over (partition by Store_id order by Date) as Sales_Lag_30,
    avg(Sales) over (partition by Store_id order by Date rows between 7 preceding and 1 preceding) as Sales_Mean_7,
    avg(Sales) over (partition by Store_id order by Date rows between 12 preceding and 1 preceding) as Sales_Mean_12,
    avg(Sales) over (partition by Store_id order by Date rows between 30 preceding and 1 preceding) as Sales_Mean_30,
    stddev_pop(Sales) over (partition by Store_id order by Date rows between 7 preceding and 1 preceding) as Sales_Std_7,
    stddev_pop(Sales) over (partition by Store_id order by Date rows between 12 preceding and 1 preceding) as Sales_Std_12,
    stddev_pop(Sales) over (partition by Store_id order by Date rows between 30 preceding and 1 preceding) as Sales_Std_30
    from train
    order by Store_id, Date
""").df()

# # Create lag features
# df['Sales_Lag_1'] = df.groupby('Store_id')['Sales'].shift(1)
# df['Sales_Lag_7'] = df.groupby('Store_id')['Sales'].shift(7)
# df['Sales_Lag_12'] = df.groupby('Store_id')['Sales'].shift(12)
# df['Sales_Lag_30'] = df.groupby('Store_id')['Sales'].shift(30)

# # Create moving average features
# df['Sales_Mean_7'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=7).mean()
# df['Sales_Mean_12'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=12).mean()
# df['Sales_Mean_30'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=30).mean()

# # Create moving standard deviation features
# df['Sales_Std_7'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=7).std()
# df['Sales_Std_12'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=12).std()
# df['Sales_Std_30'] = df.groupby('Store_id')['Sales'].shift(1).rolling(window=30).std()

# # Sort the DataFrame by Store_id and Date
# df = df.sort_values(by=['Store_id', 'Date'])

In [ ]:
ohe = OneHotEncoder(categories='auto')
feature_array = ohe.fit_transform(train[['Store_Type', 'Location_Type', 'Region_Code','cluster']]).toarray()
train = pd.concat([train, pd.DataFrame(feature_array, columns=ohe.get_feature_names_out())], axis=1).drop(['Store_Type', 'Location_Type', 'Region_Code', 'cluster'], axis=1)
train = train.drop(columns=["Date","Store_id"])
train

,Store_id,Holiday,Discount,Sales,Day,Date,Day_of_Week,Month,Quarter,Week,...,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,cluster_1,cluster_2,cluster_3,cluster_4
0,1,1,1,7011.84,1,2018-01-01,0,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,1,42369.00,2,2018-01-02,1,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0,1,50037.00,3,2018-01-03,2,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0,1,44397.00,4,2018-01-04,3,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0,1,47604.00,5,2018-01-05,4,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150740,365,0,0,19653.00,13,2019-02-13,2,2,1,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
150741,365,0,1,33711.00,14,2019-02-14,3,2,1,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
150742,365,0,1,29061.00,15,2019-02-15,4,2,1,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
150743,365,0,1,40164.00,16,2019-02-16,5,2,1,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [67]:
train.columns

Index(['Holiday', 'Discount', 'Sales', 'Day', 'Day_of_Week', 'Month',
       'Quarter', 'Week', 'Week_of_Month', 'Is_Weekend', 'Sales_Lag_1',
       'Sales_Lag_7', 'Sales_Lag_12', 'Sales_Lag_30', 'Sales_Mean_7',
       'Sales_Mean_12', 'Sales_Mean_30', 'Sales_Std_7', 'Sales_Std_12',
       'Sales_Std_30', 'Store_Type_S1', 'Store_Type_S2', 'Store_Type_S3',
       'Store_Type_S4', 'Location_Type_L1', 'Location_Type_L2',
       'Location_Type_L3', 'Location_Type_L4', 'Location_Type_L5',
       'Region_Code_R1', 'Region_Code_R2', 'Region_Code_R3', 'Region_Code_R4',
       'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4'],
      dtype='object')